In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from scipy.special import inv_boxcox

In [2]:
df = pd.read_csv('data/NBA_Player_Dataset-2013-2021.csv')

box_cox_transformer = 0.16106145323461019 #need to pull from data_prep.ipynb

In [3]:
#Can be used to filter out earlier years
#df = df[~df['Year'].isin([2013, 2014, 2015])]

In [4]:
df.columns

Index(['Year', 'FULL NAME', 'TEAM', 'POS', 'AGE', 'GP', 'MPG', 'MIN%', 'USG%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'VI', 'ORTG', 'DRTG',
       'TO_100_Games', 'Salary', 'Cap Maximum', 'Salary%OfCap', 'Traded',
       'Center', 'Forward', 'Guard', 'Salary_Scaled', 'Salary_BoxCox',
       'Salary_Scaled_BoxCox'],
      dtype='object')

In [5]:
#variables we need to scale

x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'MIN%', 'USG%',
       'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG', 'RPG',
       'TRB%', 'APG', 'AST%', 'SPG', 'BPG', 'VI', 'TO_100_Games']]

In [6]:
'''x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'PPG', 'RPG',
       'APG', 'SPG', 'BPG']]'''

"x_to_scale = df[['Year', 'AGE', 'GP', 'MPG', 'PPG', 'RPG',\n       'APG', 'SPG', 'BPG']]"

In [7]:
#scaler function
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_to_scale)

x_scaled = scaler.transform(x_to_scale)

x_scaled_df = pd.DataFrame(x_scaled, columns = x_to_scale.columns)

In [8]:
x_scaled_df

,Year,AGE,GP,MPG,MIN%,USG%,FTA,FT%,2PA,2P%,...,TS%,PPG,RPG,TRB%,APG,AST%,SPG,BPG,VI,TO_100_Games
0,-0.448291,1.054938,0.482581,1.246624,1.249235,1.006838,1.273851,0.917979,0.808159,0.245669,...,0.789723,1.547901,0.546992,-0.257996,4.043414,3.665499,3.171747,-0.651949,2.259476,0.178108
1,0.324087,-0.783816,-1.268083,-0.180363,-0.172433,0.394914,-0.644776,0.148347,-0.722653,-0.567050,...,-0.274237,-0.179525,-0.789344,-0.935353,-0.554199,-0.596403,-0.175151,-0.436866,-0.732954,-0.095976
2,-1.606858,0.345907,0.930425,0.316920,0.299727,0.272529,0.214435,1.150321,0.498943,-0.306664,...,0.012569,0.130108,-0.708354,-1.032118,0.775231,1.023554,-0.080204,-0.723643,0.079277,0.016882
3,1.482654,-1.067428,-0.168829,-0.104690,-0.104981,0.202595,-0.160948,0.276619,-0.337087,1.022880,...,0.729586,0.178997,1.134170,1.677307,-0.720378,-0.966058,-0.804178,0.901426,-0.027596,-0.418428
4,0.710276,-0.367851,-0.535247,-1.277630,-1.272409,-0.828935,-0.628092,-0.202586,-0.741740,1.780366,...,1.784295,-0.912867,-0.445136,1.174128,-0.720378,-0.324598,-1.053415,-0.711694,0.635014,0.823012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,-0.448291,-0.363124,1.011851,1.452024,1.461966,2.178236,3.551179,0.859894,2.495487,0.332464,...,0.752716,2.753840,0.870952,-0.064466,0.886017,0.632155,2.768220,0.805834,1.276249,-0.611899
4206,-0.062102,-0.363124,-1.064517,-1.115473,-1.111563,-0.339395,-0.819955,-1.381236,-0.848630,0.056297,...,-0.801592,-0.912867,-0.688106,0.158094,-0.526503,0.061365,-0.780441,-0.651949,0.271647,-0.757002
4207,1.482654,-1.213961,-1.064517,-1.369520,-1.365803,-0.426813,-0.794930,0.230635,-0.871534,1.665954,...,1.215308,-0.912867,-0.384393,2.006309,-0.886557,-0.694253,-0.982204,0.805834,0.335771,0.065250
4208,-1.220669,0.109564,0.441868,0.479078,0.481327,0.290013,0.439666,0.985746,0.193544,-0.211979,...,0.179102,0.407148,-0.789344,-1.206295,-0.332627,-0.465937,0.821797,-0.723643,-0.775703,-0.692512


In [22]:
#add the categorical features to scaled features for input
X = pd.concat([x_scaled_df, df[['Center', 'Forward', 'Guard']]], axis = 1)

#choose our target variable
#y = df['Salary']
#y = df['Salary_BoxCox']
y = df['Salary_Scaled_BoxCox']

# Feature Selection
SelectKBest
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

### Scorers for regression
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif

In [23]:
#Need to do

# Simple single train/test split for testing models

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

y_test_salary = inv_boxcox(y_test, box_cox_transformer)

# Random Forest

In [25]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=46)

regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

y_pred_salary = inv_boxcox(y_pred, box_cox_transformer)

mse = mean_squared_error(y_test_salary, y_pred_salary, multioutput='raw_values')
mae = mean_absolute_error(y_test_salary, y_pred_salary)
mape = mean_absolute_percentage_error(y_test_salary, y_pred_salary)

'''mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
'''

print('MSE', mse)
print('MAE', mae)
print('MAPE', mape)

MSE [6.62957379e+14]
MAE 15303999.328406708
MAPE 1.3104594754102468


In [14]:
results = pd.DataFrame(zip(*[y_test_salary,y_pred_salary]), columns = ['Y True', 'Y Tree Pred'])
results['Difference'] = abs(results['Y True'] - results['Y Tree Pred'])
results['OG Index'] = y_test.index
results

,Y True,Y Tree Pred,Difference,OG Index
0,3.591514e+06,1.474410e+06,2.117104e+06,3355
1,7.221305e+05,4.135854e+05,3.085451e+05,3124
2,7.673384e+06,8.744314e+06,1.070930e+06,1864
3,2.548640e+06,5.750574e+06,3.201934e+06,535
4,1.491075e+06,1.104638e+05,1.380611e+06,2320
...,...,...,...,...
837,1.039015e+06,3.595866e+06,2.556852e+06,3963
838,1.695634e+06,2.196851e+06,5.012167e+05,1730
839,3.519530e+06,7.046737e+06,3.527207e+06,860
840,1.692535e+06,5.098445e+06,3.405910e+06,3306


# Need to add box cox transformation to all of the below.  

# Adaboost

In [15]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=7, n_estimators=100)

regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE', mse)
print('MAE', mae)
print('MAPE', mape)

MSE [100.12066763]
MAE 8.235994979240765
MAPE 0.14835854767898735


# Linear Regression

In [17]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, y_train)

y_pred = reg.predict(X_test)

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE', mse)
print('MAE', mae)
print('MAPE', mape)

MSE [97.28057182]
MAE 8.093774866615687
MAPE 0.14437399997296113


# KNN

In [19]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=20)

neigh.fit(X_train, y_train)

y_pred = neigh.predict(X_test)

In [20]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print('MSE', mse)
print('MAE', mae)
print('MAPE', mape)

MSE [93.11279069]
MAE 7.735266037971251
MAPE 0.1395466953351569
